In [1]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interactive

In [2]:
%matplotlib qt5

In [3]:
with open('S-parameter anticrossings Rev5 Stas qubit #3.pkl', 'rb') as fhandle:
    data = pickle.load(fhandle)

In [4]:
parameter_names = data[1]['S-parameter'][0]
parameter_values = data[1]['S-parameter'][1]
measurement_result = data[1]['S-parameter'][2]

In [5]:
plt.pcolormesh(parameter_values[0], parameter_values[1], np.abs(measurement_result.T))

In [6]:
plt.pcolormesh(parameter_values[0], parameter_values[1], np.angle(measurement_result.T))

In [8]:
initial_param=[-0.013, 0.042, 3e3, 2e3, 0.1e6, 0.1e6, 4.2e6, 300e-9, 6.5862e9,6.574e9]
k=-10.5e9
br, bi, etar, etai, kappa, gamma, g, t, wr, wq0 = initial_param
w = np.reshape(parameter_values[1], (parameter_values[1].shape[0], 1))
B = np.reshape(parameter_values[0], (1, parameter_values[0].shape[0]))
wq = k*B+wq0
fun = (br+1j*bi-(etar+1j*etai)/(kappa-1j*(wr-w)+g**2/(gamma-1j*(wq-w))))*np.exp(-1j*w*t)#-measurement_result.T


plt.figure(1)
ax= plt.imshow(np.real(fun),cmap='RdBu')
plt.title("real")
plt.colorbar()

plt.figure(2)
ax2=plt.imshow(np.angle(fun),cmap='RdBu')
plt.title("imagine")
plt.colorbar()
plt.minorticks_on()

plt.figure(3)
ax3=plt.imshow(np.real(measurement_result.T),cmap='RdBu')
plt.title("real experimental")
plt.colorbar()

plt.figure(4)
ax4=plt.imshow(np.angle(measurement_result.T),cmap='RdBu')
plt.title("imagine experimental")
plt.colorbar()


def plot(x_br, x_bi, x_etar, x_etai, x_kappa, x_gamma, x_g, x_t, x_wr,x_wq):
    new = (br+x_br+1j
           *(bi+x_bi)
           -(etar+x_etar+1j*(etai+
                                 x_etai))/(kappa+x_kappa-1j*(wr+x_wr-w)+
                                                            (g+x_g)**2/(gamma+x_gamma-1j*(wq+
                                                                                         x_wq-w))))*np.exp(-1j*w*(t+x_t))
    ax.set_data(np.real(new))
    ax2.set_data(np.angle(new))

interactive(plot,
            x_wr=(-1e9,1e9, 1e6),
            x_g=(-1e6,1e6, 1e5),
            x_br=(-1e-1,1e-1, 1e-4),
            x_bi=(-1e-2,1e-2, 1e-4),
            x_etar=(-1e3,1e3, 1e2),
            x_etai=(-1e3,1e3, 1e2),
            x_kappa=(-1e5,1e5, 1e4),
            x_gamma=(-1e5,1e5, 1e4),
            x_wq=(-1e9,1e9, 1e6),
            x_t=(-100e-9,100e-9, 10e-9),
           )

interactive(children=(FloatSlider(value=0.0, description='x_br', max=0.1, min=-0.1, step=0.0001), FloatSlider(value=0.0, description='x_bi', max=0.01, min=-0.01, step=0.0001), FloatSlider(value=0.0, description='x_etar', max=1000.0, min=-1000.0, step=100.0), FloatSlider(value=0.0, description='x_etai', max=1000.0, min=-1000.0, step=100.0), FloatSlider(value=0.0, description='x_kappa', max=100000.0, min=-100000.0, step=10000.0), FloatSlider(value=0.0, description='x_gamma', max=100000.0, min=-100000.0, step=10000.0), FloatSlider(value=0.0, description='x_g', max=1000000.0, min=-1000000.0, step=100000.0), FloatSlider(value=0.0, description='x_t', max=1e-07, min=-1e-07, step=1e-08), FloatSlider(value=0.0, description='x_wr', max=1000000000.0, min=-1000000000.0, step=1000000.0), FloatSlider(value=0.0, description='x_wq', max=1000000000.0, min=-1000000000.0, step=1000000.0), Output()), _dom_classes=('widget-interact',))